In [18]:
import numpy as np
import pandas as pd
import os
from functions import load_monitor_data
from functions import export_to_vtp

np.random.seed(23765)

### Set parameters

In [19]:
user = 'Kattelijn'
simulation = 'disable' # 'no-network', 'disable', 'stimulus', 'calcium' 
number_of_neurons = 50000
time_steps = np.arange(50000, 150100, 100) # 1,000,000 simulation steps available, every 100th iteration step is sampled
monitor_variables = ['fired_fraction', 'electric_activity'] # 'fired', 'fired_fraction', 'electric_activity', 'secondary_variable', 'calcium', 'target_calcium', 'synaptic_input', 'background_activity', 'grown_axons', 'connected_axons', 'grown_excitatory_dendrites', 'connected_excitatory_dendrites'
process_by_area = True # use for large numbers of neurons and/or monitor variables

### Load positions data

In [20]:
if user == 'Kattelijn':
    base_folder = f'/Volumes/My Passport/viz-{simulation}'
elif user == 'Kevin':
    base_folder = f'/Users/Kevin/Documents/Opleiding/Master/Computational_Science/Year_2/Semester_1/Period_2/Scientific_Visualisation_and_Virtual_Reality/Project/SciVisContest23/SciVisContest23/viz-{simulation}/'

In [ ]:
column_headers_pos = ['local id', 'pos x', 'pos y', 'pos z', 'area', 'type']

df_pos = pd.read_csv(os.path.join(base_folder, 'positions/rank_0_positions.txt'), skiprows= 8, usecols=[0, 1, 2, 3, 4], delimiter=' ', names=column_headers_pos)

neurons = np.sort(np.random.choice(np.arange(0, 50000, 1), size=number_of_neurons, replace=False))

df_pos_filtered = df_pos[df_pos['local id'].isin(neurons)]

### Process all neurons at once

In [23]:
if not process_by_area:
    if len(monitor_variables) >= 11:
        output_folder = f"vtp_files/{simulation}/{number_of_neurons}_neurons/all_neurons/all_variables"
    else:
        output_folder = f"vtp_files/{simulation}/{number_of_neurons}_neurons/{monitor_variables}"

    if os.path.exists(output_folder):
        response = input(f"Warning: The folder '{output_folder}' already exists. Overwrite existing files? (yes/no): ")
        if response.strip().lower() != 'yes':
            exit()

    time_steps_set = set(time_steps)

    df_monitor = load_monitor_data(neurons, time_steps_set, monitor_variables, base_folder)

    for time_point in time_steps:

        df_monitors_at_time = df_monitor[df_monitor['step'] == time_point]

        df_merged = pd.merge(df_pos_filtered, df_monitors_at_time, left_on='local id', right_on='neuron_id')

        filename = f'{simulation}_timestep_{time_point}.vtp'
        export_to_vtp(df_merged.to_dict('records'), output_folder, filename, monitor_variables)

        print(f"Saved: {os.path.join(output_folder, filename)}")

### Process neurons by area

In [ ]:
if process_by_area:
    area_groups = df_pos_filtered.groupby('area')['local id'].apply(list).to_dict()
    
    if len(monitor_variables) >= 11:
        output_folder = f"vtp_files/{simulation}/{number_of_neurons}_neurons/by_area/all_variables"
    else:
        output_folder = f"vtp_files/{simulation}/{number_of_neurons}_neurons/by_area/{monitor_variables}"

    if os.path.exists(output_folder):
        response = input(f"Warning: The folder '{output_folder}' already exists. Overwrite existing files? (yes/no): ")
        if response.strip().lower() != 'yes':
            exit()

    time_steps_set = set(time_steps)

    for area, neurons_in_area in area_groups.items():
        try:
            print(f"Processing {area}")
            
            df_monitor = load_monitor_data(neurons_in_area, time_steps_set, monitor_variables, base_folder, area)
            df_merged = pd.merge(df_monitor, df_pos_filtered, left_on='neuron_id', right_on='local id', how='inner')
            
            os.makedirs(output_folder, exist_ok=True)

            for time_point in time_steps:
                print(f"Writing VTP for area {area} at timestep {time_point}...")
                df_timestep = df_merged[df_merged['step'] == time_point]

                filename = f"{simulation}_{area}_timestep_{time_point}.vtp"
                export_to_vtp(df_timestep.to_dict('records'), output_folder, filename=filename, monitor_variables=monitor_variables)
                print(f"Saved: {os.path.join(output_folder, filename)}")
            
            print(f"Completed {area}")
        except Exception as e:
            print(f"Error processing {area}: {e}")